In [2]:
import itertools
import numpy as np

from misc import factors
from misc.factors import factor_count, factor_list
from solver import EquationParser


def create_factor_grid():
    grid = np.zeros((100, 100), dtype=int)
    for i, j in itertools.combinations(range(100), 2):
        grid[i, j] = grid[j, i] = factor_count(abs(i * i - j * j))
    return grid

grid = create_factor_grid()


In [3]:
TAGS = """AC AD BF
AH DE DG EH
BG CH
CE EG
AE
CD CF DH"""
lines = TAGS.splitlines()
lines = [line.split() for line in lines]

all_items = [item for line in lines for item in line]


In [90]:
"""
line1 AC AD BF            ac, ad
line2 AH DE DG EH         ah, de, eh, dg
line3 BG CH               ch
line4 CE EG               ce
line5 AE                  ae
line6 CD CF DH            cd, dh, cf

ACDEGH   BFG

bf,
bg

"""
def f(x, y):
    return factor_count(abs(x * x - y * y))

def run():
    count = 0
    for D, H in zip(*np.where(grid == 8)):
        for E in range(1, 100):
            if E in (D, H): continue
            line2 = grid[E, D]
            if not line2 <= 8: continue
            if  grid[E, H] != line2: continue
            for C in range(1, 100):
                if C in (D, E, H): continue
                if grid[C, D] != 8: continue
                line3 = grid[C, H]
                line4 = grid[C, E]
                if not(line2 <= line3 <= line4 <= 8): continue
                for A in range(1, 100):
                    if A in (C, D, E, H): continue
                    line1 = grid[A, C]
                    line5 = grid[A, E]
                    if not (line1 < line2 < line3 < line4 < line5 < 8): continue
                    if grid[A, D] != line1: continue
                    if grid[A, H] != line2: continue
                    for G in range(1, 100):
                        if G in (A, C, D, E, H): continue
                        if grid[D, G] != line2: continue
                        if grid[E, G] != line4: continue
                        for F in range(1, 100):
                            if F in (A, C, D, E, G, H): continue
                            if grid[C, F] != 8: continue
                            for B in range(1, 100):
                                if B in (A, C, D, E, G, H, F): continue
                                if grid[B, F] != line1: continue
                                for G in range(1, 100):
                                    if G in (A, B, C, D, E, F, H): continue
                                    if grid[B, G] != line3: continue
                                    count += 1
                                    print(f"({A} {B} {C} {D} {E} {F} {G} {H}) {grid[A, C], grid[A, D], grid[B, F]} / "
                                          f"{grid[A, H], grid[D,E], grid[D, G], grid[E, H]} / "
                                          f"{grid[B, G], grid[C, H]} /"
                                          f"{grid[C, E], grid[E, G]} / {grid[A, E]} / {grid[C, D], grid[C, F], grid[D, H]}")

    return count

def run2():
   lookup = dict(zip("ABCDEFGH", (6, 15, 5, 7, 10, 16, 12, 3)))
   pairs = "AG, AH, BG, CD, DE, DF".split(', ')
   for l1, l2 in pairs:
       value1, value2 = lookup[l1], lookup[l2]
       result = abs(value1 * value1 - value2 * value2)
       assert result == (value1 + value2) * abs(value1 - value2)
       print(f"{l1}{l2} = ({value1},{value2}) = {result}")



AG = (6,12) = 108
AH = (6,3) = 27
BG = (15,12) = 81
CD = (5,7) = 24
DE = (7,10) = 51
DF = (7,16) = 207


In [5]:

from solver import Clue, EquationSolver, Evaluator, Location



class Magpie269 (EquationSolver):
    @classmethod
    def run(cls):
        solver = cls()
        solver.solve(debug=100)

    def __init__(self) -> None:
        clues = self.get_clues()
        super().__init__(clues, items=range(1, 100))
        for clue in clues:
            (row, _, _) = clue.context
            if row < 5:
                self.add_constraint((clue,), lambda x: int(x) < 8)
            else:
                self.add_constraint((clue,), lambda x: int(x) == 8)
        for clue1, clue2 in itertools.combinations(clues, 2):
            (row1, _, _) = clue1.context
            (row2, _, _) = clue2.context
            if row1 == 5 or row2 == 5: continue
            if row1 < row2:
                self.add_constraint((clue,), lambda x, y: int(x) < int(y))
            if row1 == row2:
                self.add_constraint((clue,), lambda x, y: int(x) == int(y))
            if row1 > row2:
                self.add_constraint((clue,), lambda x, y: int(x) > int(y))

    def get_clues(self):
        clues = []
        for row, line in enumerate(lines):
            for letter1, letter2 in line:
                clue_number = len(clues) + 1
                expression = f"  \"grid\" [ {letter1}, {letter2} ]"
                clue = Clue(f"{clue_number}a", True, (clue_number, 1), 1, context=(row, letter1, letter2))
                clue.evaluators = Evaluator.create_evaluators(expression, mapping={'grid': grid})
                clues.append(clue)
        for clue in clues:
            print(clue.context)
        return clues

    def get_allowed_regexp(self, location: Location) -> str:
        return ".*"


Magpie269.run()


Parsing f  "grid" [ A, C ]
Parsing f  "grid" [ A, D ]
Parsing f  "grid" [ B, F ]
Parsing f  "grid" [ A, H ]
Parsing f  "grid" [ D, E ]
Parsing f  "grid" [ D, G ]
Parsing f  "grid" [ E, H ]
Parsing f  "grid" [ B, G ]
Parsing f  "grid" [ C, H ]
Parsing f  "grid" [ C, E ]
Parsing f  "grid" [ E, G ]
Parsing f  "grid" [ A, E ]
Parsing f  "grid" [ C, D ]
Parsing f  "grid" [ C, F ]
Parsing f  "grid" [ D, H ]
(0, 'A', 'C')
(0, 'A', 'D')
(0, 'B', 'F')
(1, 'A', 'H')
(1, 'D', 'E')
(1, 'D', 'G')
(1, 'E', 'H')
(2, 'B', 'G')
(2, 'C', 'H')
(3, 'C', 'E')
(3, 'E', 'G')
(4, 'A', 'E')
(5, 'C', 'D')
(5, 'C', 'F')
(5, 'D', 'H')
<Clue 5a> ('D', 'E')
<Clue 10a> ('C',)
<Clue 13a> ()
<Clue 1a> ('A',)
<Clue 2a> ()
<Clue 12a> ()
<Clue 4a> ('H',)
<Clue 7a> ()
<Clue 9a> ()
<Clue 15a> ()
<Clue 6a> ('G',)
<Clue 11a> ()
<Clue 8a> ('B',)
<Clue 3a> ('F',)
<Clue 14a> ()
 5a letters=('D', 'E') pattern=".*"
 5a DE (1, 2) 2 (1): -->
 |  10a letters=('C',) pattern=".*"
 |  10a C (4,) 6 (1): -->
 |  |  13a letters=() pattern